<h1>Recurrent Neural Network</h1>

# 0. Recurrent Neural Network

考虑典型的循环神经网络，考虑输入长度为$\tau$的时间序列。

+ 节点

> 输入节点：$x^{(t)} \in \mathbb{R}^{batch\_size \times n\_input\_unit}, t=1, 2, ..., \tau$

> 输出节点：$\hat{y}^{(t)} \in \mathbb{R}^{batch\_size \times n\_output\_unit}$

> 隐含节点：$h^{(t)} \in \mathbb{R}^{n\_input\_unit \times n\_output\_unit}$

+ 网络连接权重

> 输入到隐含节点：$U \in \mathbb{R}^{n\_input\_unit \times n\_hidden\_unit}$

> 隐含到输出节点：$V \in \mathbb{R}^{n\_hidden\_unit \times n\_output\_unit}$

> 隐含到隐含节点: $W \in \mathbb{R}^{n\_hidden\_unit \times n\_hidden\_unit}$

# 1. Forward Propagation

对于$t \in \{1, 2, ..., \tau\}$，依次计算$h^{(t)}$及$\hat{y}^{(t)}$，

+ 节点计算规则

> 隐含节点输入: $a^{(t)} = Wh^{(t-1)} + Ux^{(t)} + b$

> 隐含节点输出: $h^{(t)} = tanh(a^{(t)})$

> 输入节点输入: $o^{(t)} = Vh^{(t)} + c$

> 输出节点输出: $\hat y^{(t)} = softmax(o^{(t)})$ 

# 2. Back Propagation Error

## 2.0 损失函数
交叉熵cross-entropy。考虑包含$N$个样本的mini-batch平均误差，每个样本的误差是所有$\tau$个时间序列的误差之和；

> $L_{mini\_batch} = \frac{1}{N}\sum_{n=1}^{N}L_n$

> $L_n = \sum_{t=1}^{\tau} L_n^{(t)}$

> $L_n^{(t)} = - y_n^{(t)} \log \hat{y}_n^{(t)}$

## 2.1 对权重$U,V,W$的偏导
对于$t \in \{\tau, \tau - 1, ..., 2, 1\}$，依次计算mini_batch的损失$L_{mini\_batch}$对各个权重$U$、$V$以及$W$的偏导，

> $\frac{\partial{L}}{\partial{U_{ij}}} 
= \frac{1}{N} \sum_{n=1}^{N}\frac{\partial{L_n}}{\partial{U_{ij}}}$

所以，只需要计算每个样本$L_n$对$U$、$V$以及$W$的偏导，然后求均值即可。为书写清洁，将$L_n$简写为$L$。

这里偏导、梯度、转置的关系有待进一步理解。

> $\nabla_{c}{L} = \sum_{t}\left( 
\frac{\partial{o^{(t)}}}{\partial{c}} \right) ^ {T}
\nabla_{o^{(t)}}{L}$

> $\nabla_{b}{L} = \sum_{t}\left( 
\frac{\partial{h^{(t)}}}{\partial{b}} \right) ^ {T} \nabla_{h^{(t)}}{L}
= \sum_t diag \left( 1 - (h^{(t)})^2 \right) \nabla_{h^{(t)}}{L}$ 

> $\nabla_{V}{L} = \sum_{t} \sum_{i}
\left(  \frac{\partial{L}}{\partial_{o_i^{(t)}}} \right) \nabla_{V}{o_i^{(t)}} 
= \sum_t \left( \nabla_{V}{o^{(t)}}\right) {h^{(t)}}^{T} $

> $\nabla_{W}{L} = \sum_t \sum_i
\left( \frac{\partial{L}}{\partial{h_i^{(t)}}} \right) \nabla_{W}{h_i^{(t)}}
=\sum_t diag \left( 1 - (h^{(t)})^2 \right) (\nabla_{h^{t}}{L}) {h^{(t-1)}}^T$

> $\nabla_{U}{L} = \sum_t \sum_i
\left( \frac{\partial_{L}}{\partial{h_i^{(t)}}} \right) \nabla_{U}{h_i^{(t)}}
= \sum_t diag \left( 1 - (h^{(t)})^2 \right) (\nabla_{h^{t}}{L}) {x^{(t)}}^T$




### 2.1.0 计算并保存$\nabla_{o^{(t)}}L$及$\nabla_{h^{(t)}}L$

softmax求导;
> $f(x_i) = \frac{e^{x_i}}{\sum_{j}e^{x_j}}$

> $\frac{\partial{f(x_i)}}{\partial{x_i}} = 
e^{x_i} * \left(\frac{1}{\sum_{j}e^{x_j}}\right) + 
e^{x_i} * \left(- \frac{1}{\left(\sum_{j}e^{x_j}\right)^2} * e^{x_i}\right)$

> $ = \frac{e^{x_i}}{\sum_{j}e^{x_j}} \left( 1 - \frac{e^{x_i}}{\sum_{j}e^{x_j}} \right)$

所以softmax的导数与sigmoid一样的形式，
> $f^{'}(x_i) = f(x_i) \left(1 - f(x_i)\right)$

#### 2.1.0.0 计算$\nabla_{o^{(t)}}L$

> $\left(\nabla_{o^{(t)}}L\right)_i = \frac{\partial{L}}{\partial{L^{(t)}}}
\frac{\partial{L_t}}{\partial{o_i^{(t)}}}$

> $ = 1 * \frac{\partial{L_t}} {\partial{\hat{y_i}^{(t)}}}
\frac{\partial{\hat{y_i}^{(t)}}} {\partial{o_i^{(t)}}} $

> $ = \left( -\frac{y_i^{(t)}}{\hat{y_i}^{(t)}} \right)
\left( \hat{y_i}^{(t)} (1 - \hat{y_i}^{(t)})\right)$

> $ = (\hat{y_i}^{(t)} - 1) y_i^{(t)}$

> $\nabla_{o^{(t)}}L = (\hat{y}^{(t)} - 1) y^{(t)}$

#### 2.1.0.1 计算$\nabla_{h^{(t)}}L$

+ 最后一个时间步，$h^{(\tau)}$的误差只受$o^{(\tau)}$的误差影响；

> $\left(\nabla_{h^{(\tau)}}L \right)_i = 
\frac{\partial{L}}{\partial{o_i^{(\tau)}}}$

> $\nabla_{h^{(\tau)}}L = \frac{\partial{o_i^{(\tau)}}}{\partial{h_i^{(\tau)}}}
= V^{T} \nabla_{o^{(\tau)}}L$

+ 时间步$t = 0,1,...,\tau-1$，$h^{(t)}$的误差受$o^{(t)}$的误差以及$h^{(t+1)}$的误差影响；

> $\left(\nabla_{h^{(t)}}L \right)_i = 
\frac{\partial{L}}{\partial{o_i^{(t)}}}
\frac{\partial{o_i^{(t)}}}{\partial{h_i^{(t)}}} + 
\frac{\partial{L}}{\partial{h_i^{(t+1)}}}
\frac{\partial{h_i^{(t+1)}}}{\partial{h_i^{(t)}}}$

> $\nabla_{h^{(t)}}L = 
V^{T} \nabla_{o^{(t)}}L +
W^{T} \left(\nabla_{h^{(t+1)}}L\right) diag \left( 1 - h^{(t+1)} \right)^2$ 

# 3. Update Weights

对于mini-batch，

> $U_{ij} = U_{ij} - \eta_u * \frac{\partial{L}}{\partial{U_{ij}}}$

> $V_{ij} = V_{ij} - \eta_v * \frac{\partial{L}}{\partial{V_{ij}}}$

> $W_{ij} = W_{ij} - \eta_w * \frac{\partial{L}}{\partial{W_{ij}}}$